<a href="https://colab.research.google.com/github/arkB/podcast_short_subtitles/blob/main/podcast_short_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload short mp3
from google.colab import files
upload_files = files.upload()

Saving Ep89_5_1-short.mp3 to Ep89_5_1-short.mp3
Saving arkbfm.png to arkbfm.png


In [ ]:
!pip install faster_whisper
!pip install srt
!pip install budoux
!apt install -y fonts-takao
!cp *.ttf "/usr/share/fonts/truetype/"
!fc-cache -fv
apt install libcublas11

In [3]:
# Add line breaks to subtitles
def combine_text_until_max_length(data, max_length):
    if not data:
        return []

    combined_text = ""
    current_string = ""

    for item in data:
        if len(current_string + item) <= max_length:
            current_string += item
        else:
            combined_text += current_string + "\n"
            current_string = item

    if current_string:
        combined_text += current_string + "\n"

    return combined_text

In [4]:
# Transcribe and create .srt subtitle file
import budoux
from faster_whisper import WhisperModel
from datetime import timedelta
from srt import Subtitle
import srt
import glob

files = glob.glob(r"*.mp3", recursive=True)
file_name = files[0].rstrip(".mp3")
file_path = file_name + ".mp3"
image_files = glob.glob(r"*.png", recursive=True)
image_file = image_files[0]
model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe(file_path, beam_size=10)
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

parser = budoux.load_default_japanese_parser()

subs = []
for segment in segments:
  print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
  index = int(segment.id) + 1
  start = segment.start
  end = segment.end
  segment_text = segment.text.replace("、", " ")
  segment_text = segment_text.replace("。", "")
  word_list = parser.parse(segment_text)
  text = combine_text_until_max_length(word_list, 25)
  sub = Subtitle(index=1, start=timedelta(seconds=timedelta(seconds=start).seconds,
                                          microseconds=timedelta(seconds=start).microseconds),
                                          end=timedelta(seconds=timedelta(seconds=end).seconds,
                                          microseconds=timedelta(seconds=end).microseconds), content=text, proprietary='')
  subs.append(sub)

with open(file_name + ".srt", mode="w", encoding="utf-8") as f:
    f.write(srt.compose(subs))

Detected language 'ja' with probability 0.997070
[0.00s -> 10.40s] アファマティブアクション、前回の繰り返りの時も何かの話題で、東高大のこれは私は関心しないという話はしたと思うんですけれども
[10.40s -> 18.00s] そうですね、関心しない。これがノーマライゼーション、ノーマルの状態がなんだかよくわかんないっていうのもありますけれども
[18.00s -> 28.00s] つまりずっとこれが100年、200年、300年、日本が続いて東高大産も筒がなくお続きになって
[28.00s -> 36.80s] 人類も成熟して人権があれして、というような時代が続いた時に合体語になっているのかどうかっていうのはよくわからない
[36.80s -> 44.00s] 結局やっぱり今と男女比はもしかして変わらないのかもしれないし、男女逆転しているのかもしれないし性別がなくなっているかもしれませんけど
[45.00s -> 58.00s] 何に向かってどういう合体語が正しいっていうことがあまり根拠はないんじゃないのっていうのもありますね
[58.00s -> 67.00s] ちょっとなんかひどい垂れ流しになってしまって、適当につまんでほしいな、もういらないとこしっかり言ってほしいな、もうごめんね
[68.00s -> 76.00s] そんな感じでちょっと難しいタイムラインについて4、5人ぐらいで哲学対話という形でちょっと話したよっていう対話がありました
[76.00s -> 85.00s] ただ私が排出するのはゴミですけれども大変私には資産に富んだ面白い回でした、本当に
[85.00s -> 92.00s] 私が排出するのはゴミとしてもとても役に立ちました、私の人生ではね
[92.00s -> 96.00s] ちょっと今後も注視していこうと思います、その方面の議論はね


In [ ]:
# Create embedded subtitle file .ass
!ffmpeg -i $file_name".srt" $file_name".ass"

In [ ]:
# Create a single picture video file .mp4 without subtitles
!ffmpeg \
    -loop 1 \
    -r 30000/1001 \
    -i $image_file -i $file_name".mp3" \
    -vcodec libx264 \
    -acodec aac -strict experimental -ab 320k -ac 2 -ar 48000 \
    -pix_fmt yuv420p \
    -shortest \
    $file_name".mp4"

In [ ]:
# Create a short video file .mp4 with subtitles
!ffmpeg \
    -i $file_name".mp4" \
    -vf "subtitles=$file_name'.ass':'force_style=FontName=TakaoPGothic,Fontsize=22,Bold=3,PrimaryColour=&H00CAF8F9,SecondaryColour=&H00FFFFFF,OutlineColour=&H00313131,Outline=2'" \
    $file_name"_subtitles.mp4"

In [12]:
from google.colab import files
download_files = files.download(file_name + "_subtitles.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>